In [7]:
import os
from dotenv import load_dotenv

In [8]:
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")


In [9]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

In [10]:
arxiv_api = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=300)
wiki_api = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)

In [11]:
arxiv = ArxivQueryRun(api_wrapper=arxiv_api)
wikipedia = WikipediaQueryRun(api_wrapper=wiki_api)

In [12]:
from langchain_community.document_loaders import WebBaseLoader